This class represents a dataset of training samples. It provides methods to gather samples from a specified folder, convert the data into tensors, and calculate R-squared scores for model predictions. It also provides functions to perform training and testing of models using the gathered samples.


In [5]:
import import_ipynb
import os
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import torch
import pickle
from sklearn.model_selection import train_test_split
import model_creator
import model_training
from training_sample import training_sample
import locations
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import locations as l
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter



class training_dataset:
    def __init__(self, dataset_folder, col):
        """
        Initializes the training_dataset object by gathering training samples from the specified dataset folder.

        Parameters:
        - dataset_folder: str, the path to the folder containing the dataset files.
        - col: list, the columns to be used for processing the dataset.
        """
        self.samples = self.gather_samples(dataset_folder, col)


    def gather_samples(self, dataset_folder, col):
        """
        Collects all training samples from the specified dataset folder and initializes them as training_sample objects.

        Parameters:
        - dataset_folder: str, the path to the folder containing the dataset files.
        - col: list, the columns to be used for processing each sample.

        Returns:
        - list: A list of training_sample objects created from the files in the folder.
        """
        all_items = os.listdir(dataset_folder)
        files = [item for item in all_items if os.path.isfile(os.path.join(dataset_folder, item))]
        samples = []
        for i in files:
            sample = training_sample(os.path.join(dataset_folder, i),col)
            samples.append(sample)

        return samples

    def return_as_tensors_split(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T']):
        """
        Converts the dataset into tensors and splits it into training and testing sets.
        
        Parameters:
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        
        Returns:
        - list: A list containing tensors [x_train, y_train, x_test, y_test].
        """
        df = pd.DataFrame()
    
        for sample in self.samples:
            df = pd.concat([df, sample.data[feature_columns + target_columns]], ignore_index=True)

        
        df.columns = feature_columns + target_columns
        
        features = df[feature_columns]
        targets = df[target_columns]
    
        x_train, x_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)
    
        x_train = torch.from_numpy(x_train.to_numpy(dtype=np.float32))
        x_test = torch.from_numpy(x_test.to_numpy(dtype=np.float32))
        y_train = torch.from_numpy(y_train.to_numpy(dtype=np.float32))
        y_test = torch.from_numpy(y_test.to_numpy(dtype=np.float32))
    
        return [x_train, y_train, x_test, y_test]

    def return_as_tensors(self, columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns = ['T']):
        """
        Converts the entire dataset into tensors without splitting.
        
        Parameters:
        - columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        
        Returns:
        - list: A list containing tensors [features, targets].
        """
        df = pd.DataFrame()

        for sample in self.samples:
            df = pd.concat([df, sample.data[columns + target_columns]], ignore_index=True)

        df.columns = columns + target_columns
        features = df[columns]
        targets = df[target_columns]
        features = torch.from_numpy(features.to_numpy(dtype=np.float32))
        targets = torch.from_numpy(targets.to_numpy(dtype=np.float32))
        return [features, targets]

    def get_values(self, value = ["T"]):
        """
        Extracts specific values from the dataset samples.
        Parameters:
        - value: list, the name(s) of the column(s) to extract values from. Default is ["T"].
        Functionality:
        - Iterates through all the samples in the dataset.
        - For each sample, retrieves the first value from the specified column(s).
        - Converts the value to a float and appends it to a list.
        - Returns the list of extracted values.
        Example:
        If the dataset contains multiple samples with a column "T", this function will extract
        the first value from the "T" column of each sample and return the resulting list.
        """
        data = []
        for sample in self.samples:
            data.append(sample.data[value].iloc[:1].reset_index(drop=True).astype(float).values.tolist()[0][0])
        
        
        return(data)



    def get_total_r2_score(self, model, features = ['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], targets = ['T']):
        """
        Calculates the total R-squared score for the model's predictions on the entire dataset.
        
        Parameters:
        - model: str, the name of the model to evaluate.
        - features: list, the names of the feature columns to include.
        - targets: list, the names of the target columns to include.
        
        Returns:
        - float: The total R-squared score.
        """
        model = model_creator.MLP.create_and_load(model, input_size=len(features), output_size=len(targets))
        model.eval()
        data = self.return_as_tensors(features, targets)
        features = data[0]
        targets = data[1]
        with torch.no_grad():
            predictions = model(features)
            predictions = predictions.flatten().tolist()

        pearson = pearsonr(predictions, targets.flatten().tolist())
        r2_score = pearson[0] ** 2
        return float(r2_score)

    def get_median_r2_score(self, model, features = ['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], targets = ['T']):
        """
        Calculates the median R-squared score for the model's predictions using median values of predictions for each file.
        
        Parameters:
        - model: str, the name of the model to evaluate.
        - features: list, the names of the feature columns to include.
        - targets: list, the names of the target columns to include.
        
        Returns:
        - float: The median R-squared score.
        """
        
        medians = []
        data = []
        

        for sample in self.samples:
            medians.append(sample.predict_median(model, features, len(targets)))
            dataDummy = sample.data[targets] 
            dataDummy = dataDummy.iloc[:1].reset_index(drop=True)
            dataDummy = dataDummy.astype(float).values.tolist()
            dataDummy = dataDummy[0][0]
            data.append(dataDummy)            

        pearson = pearsonr(medians, data)
        r2_score = pearson[0] ** 2
        
        
        return float(r2_score)

    def get_mean_r2_score(self, model, features = ['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], targets = ['T']):
        """
        Calculates the mean R-squared score for the model's predictions using mean values of predictions for each file.
        
        Parameters:
        - model: str, the name of the model to evaluate.
        - features: list, the names of the feature columns to include.
        - targets: list, the names of the target columns to include.
        
        Returns:
        - float: The mean R-squared score.
        """
        means = []
        data = []
        

        for sample in self.samples:
            means.append(sample.predict_mean(model, features, len(targets)))
            dataDummy = sample.data[targets] 
            dataDummy = dataDummy.iloc[:1].reset_index(drop=True)
            dataDummy = dataDummy.astype(float).values.tolist()
            dataDummy = dataDummy[0][0]
            data.append(dataDummy)            
        #print (means)
        #print (data)
        pearson = pearsonr(means, data)
        r2_score = pearson[0] ** 2
        return float(r2_score)

    def flat_return_as_tensors(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'],
                               target_columns=['T']):
        """
        Converts the flattened dataset into tensors.
        
        Parameters:
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        
        Returns:
        - list: A list containing tensors [features, targets].
        """
        targetDf = pd.DataFrame()
        featureDf = pd.DataFrame()
    
        for sample in self.samples:
            features, targets = sample.return_as_flat_df(feature_columns, target_columns)
            featureDf = pd.concat([featureDf, features], ignore_index=True)
            targetDf = pd.concat([targetDf, targets], ignore_index=True)

        x_train, x_test, y_train, y_test = train_test_split(featureDf, targetDf, test_size=0.2, random_state=42)
        x_train = torch.from_numpy(x_train.to_numpy(dtype=np.float32))
        x_test = torch.from_numpy(x_test.to_numpy(dtype=np.float32))
        y_train = torch.from_numpy(y_train.to_numpy(dtype=np.float32))
        y_test = torch.from_numpy(y_test.to_numpy(dtype=np.float32))
    
        return [x_train, x_test, y_train, y_test]

    def flat_return_as_tensors_1000(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'],
                               target_columns=['T']):
        """
        Converts the flattened dataset into tensors.
        
        Parameters:
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        
        Returns:
        - list: A list containing tensors [features, targets].
        """
        targetDf = pd.DataFrame()
        featureDf = pd.DataFrame()
    
        for sample in self.samples:
            features, targets = sample.return_as_flat_df_1000(feature_columns, target_columns)
            featureDf = pd.concat([featureDf, features], ignore_index=True)
            targetDf = pd.concat([targetDf, targets], ignore_index=True)

        x_train, x_test, y_train, y_test = train_test_split(featureDf, targetDf, test_size=0.2, random_state=42)
        x_train = torch.from_numpy(x_train.to_numpy(dtype=np.float32))
        x_test = torch.from_numpy(x_test.to_numpy(dtype=np.float32))
        y_train = torch.from_numpy(y_train.to_numpy(dtype=np.float32))
        y_test = torch.from_numpy(y_test.to_numpy(dtype=np.float32))
    
        return [x_train, x_test, y_train, y_test]

    def flat_return_as_tensors_100000(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'],
                               target_columns=['T']):
        """
        Converts the flattened dataset into tensors.
        
        Parameters:
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        
        Returns:
        - list: A list containing tensors [features, targets].
        """
        targetDf = pd.DataFrame()
        featureDf = pd.DataFrame()
    
        for sample in self.samples:
            features, targets = sample.return_as_flat_df_100000(feature_columns, target_columns)
            featureDf = pd.concat([featureDf, features], ignore_index=True)
            targetDf = pd.concat([targetDf, targets], ignore_index=True)

        x_train, x_test, y_train, y_test = train_test_split(featureDf, targetDf, test_size=0.2, random_state=42)
        x_train = torch.from_numpy(x_train.to_numpy(dtype=np.float32))
        x_test = torch.from_numpy(x_test.to_numpy(dtype=np.float32))
        y_train = torch.from_numpy(y_train.to_numpy(dtype=np.float32))
        y_test = torch.from_numpy(y_test.to_numpy(dtype=np.float32))
    
        return [x_train, x_test, y_train, y_test]

    def plot_histogram(self, columns=["T"], bins=10, title="Histogram", xlabel="Values", ylabel="Frequency"):
        """
        Plots a histogram based on the given list of data and adjusts axis labels automatically.

        Parameters:
        - columns: list, the name(s) of the column(s) to extract values from. Default is ["T"].
        - bins: int, the number of bins in the histogram (default is 10).
        - title: str, the title of the histogram (default is "Histogram").
        - xlabel: str, the label for the x-axis (default is "Values").
        - ylabel: str, the label for the y-axis (default is "Frequency").
        """
        data = self.get_values(columns)
        plt.figure(figsize=(8, 6))
        plt.hist(data, bins=bins, edgecolor='black')
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)

        # Automatically adjust labels to scientific notation if data range is small
        ax = plt.gca()
        ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='both', style='scientific', scilimits=(-3, 3))

        plt.show()
        

    def flat_train(self,model_name = "default",
                                feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], 
                                target_columns = ['T'], hidden_layers = [128, 64, 32], loss = nn.MSELoss(), bs = 0):
        """
        Trains a model using flattened data and saves the trained model.
        
        Parameters:
        - model_name: str, the name of the model to save.
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        - hidden_layers: list, the number of neurons in each hidden layer.
        - loss: torch.nn.Module, the loss function to use.
        - save_folder: str, the folder to save the trained model.
        """
        data = self.flat_return_as_tensors(feature_columns, target_columns)
    
        
        if model_name == "default":
            model_name = self.generate_model_name(feature_columns, target_columns, is_standardized=False, 
                                                  hidden_layers=hidden_layers, is_flat=True)
    
        model = model_creator.MLP(input_size=len(feature_columns)*71, output_size=len(target_columns), hidden_layers=hidden_layers)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
        model_training.train_model(model, loss, optimizer, data[0], data[2], data[1], data[3], model_name, batch_size=bs)

    def flat_train_standardized(self,model_name = "defaultStandarized",
                                feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], 
                                target_columns = ['T'], hidden_layers = [128, 64, 32], loss = nn.MSELoss(), bs = 0):
        """
        Trains a model using standardized flattened data and saves the trained model.
        
        Parameters:
        - model_name: str, the name of the model to save. Default is "defaultStandarized".
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        - hidden_layers: list, the number of neurons in each hidden layer.
        - loss: torch.nn.Module, the loss function to use.
        - bs: int, the batch size for training. Default is 0 (no batching).
        """

        
        data = self.flat_return_as_standardized_tensors(feature_columns, target_columns,
                                                        'scaler_wavelength_psi65_del65_psi70_del70_psi75_del75.pkl',
                                                        "scaler_T_A_B_C.pkl")

        if model_name == "defaultStandarized":
            model_name = self.generate_model_name(feature_columns, target_columns, True, hidden_layers)

        model = model_creator.MLP(input_size=len(feature_columns)*71, output_size=len(target_columns), hidden_layers=hidden_layers)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.001)
        model_training.train_model(model, loss, optimizer, data[0], data[2], data[1], data[3], model_name, batch_size=bs)


    def flat_return_as_standardized_tensors(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'],
                                            target_columns=['T'],
                                            XscalerName="ss",
                                            YscalerName="pp"):
        """
        Converts the flattened dataset into standardized tensors and splits it into training and testing sets.
        """
        targetDf = pd.DataFrame()
        featureDf = pd.DataFrame()
    
        for sample in self.samples:
            features, targets = sample.return_as_flat_df(feature_columns, target_columns)
            featureDf = pd.concat([featureDf, features], ignore_index=True)
            targetDf = pd.concat([targetDf, targets], ignore_index=True)
    
        scaler_path = l.locations.get_scalers_dir()
    
        with open(os.path.join(scaler_path, XscalerName), "rb") as f:
            x_scaler = pickle.load(f)
    
        with open(os.path.join(scaler_path, YscalerName), "rb") as f:
            y_scaler = pickle.load(f)
    
        x_scaled = x_scaler.fit_transform(featureDf)
        y_scaled = y_scaler.fit_transform(targetDf)
    
        x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=42)
        x_train = torch.from_numpy(x_train.astype(np.float32))
        x_test = torch.from_numpy(x_test.astype(np.float32))
        y_train = torch.from_numpy(y_train.astype(np.float32))
        y_test = torch.from_numpy(y_test.astype(np.float32))
    
        return [x_train, x_test, y_train, y_test]


    def create_row_scaler(self, columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75']):
        """
        Creates a StandardScaler for the specified columns in the dataset and saves it to a file.
        
        Parameters:
        - columns: list, the names of the columns to scale.
        """
        df = pd.DataFrame()

        for sample in self.samples:
            df = pd.concat([df, sample.data[columns]], ignore_index=True)

        x_scaler = StandardScaler()
        x_scaled = x_scaler.fit_transform(df)

        scalers_path = l.locations.get_scalers_dir()

        full_path = os.path.join(scalers_path, self.generate_scaler_name(columns))

        with open(full_path, "wb") as f:
            pickle.dump(x_scaler, f)

    def flat_create_scaler(self, col=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75']):
        """
        Creates a StandardScaler for the specified columns in the flattened dataset and saves it to a file.
        
        Parameters:
        - col: list, the names of the columns to scale.
        """
        df = pd.DataFrame()

        for sample in self.samples:
            features, targets = sample.return_as_flat_df(col, ['T'])
            df = pd.concat([df, features], ignore_index=True)

        x_scaler = StandardScaler()
        x_scaled = x_scaler.fit_transform(df)
        scalers_path = l.locations.get_scalers_dir()
        print(col)
        trol = col
        scalerName = self.generate_scaler_name(trol)

        full_path = os.path.join(scalers_path, scalerName)

        with open(full_path, "wb") as f:
            pickle.dump(x_scaler, f)


    def generate_scaler_name(self, columns, prefix="scaler"):
            """
            Generates a name for the scaler based on the names of the columns it scales.
        
            Args:
                columns (list): List of column names to be scaled.
                prefix (str): Optional prefix for the scaler name. Default is "scaler".
        
            Returns:
                str: Generated scaler name.
            """
            # Ensure columns is an iterable
            print(columns)

            if not isinstance(columns, (list, tuple)):
                raise TypeError("columns must be a list or tuple of column names.")
            
            # Join column names with underscores and prepend the prefix
            column_part = "_".join(columns)
            scaler_name = f"{prefix}_{column_part}.pkl"
            return scaler_name


    def train(self, model_name="default",
              feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'],
              target_columns=['T'], hidden_layers=[64, 32, 32, 16], loss=nn.MSELoss(), bs=0):
        """
        Trains a model using the dataset and saves the trained model.
    
        Parameters:
        - model_name: str, the name of the model to save.
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        - hidden_layers: list, the number of neurons in each hidden layer.
        - loss: torch.nn.Module, the loss function to use.
        - bs: int, the batch size for training. Default is 0 (no batching).
        """
        data = self.return_as_tensors_split(feature_columns, target_columns)
    
        if model_name == "default":
            model_name = self.generate_model_name(feature_columns, target_columns, is_standardized=False, 
                                                  hidden_layers=hidden_layers, is_flat=True)
            
        model = model_creator.MLP(input_size=len(feature_columns), output_size=len(target_columns), hidden_layers=hidden_layers)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
        model_training.train_model(model, loss, optimizer, data[0], data[1], data[2], data[3], model_name, batch_size=bs)

    def test(self, model_name, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T']):
        """
        Tests the model by calculating R2 coefficients for training, testing, and combined datasets.
    
        Parameters:
        - model_name: str, the name of the model to load and test.
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
    
        Returns:
        - dict: A dictionary containing R2 coefficients for training, testing, and combined datasets.
        """
        # Load the data
        data = self.return_as_tensors_split(feature_columns, target_columns)
        x_train, y_train, x_test, y_test = data
    
        # Load the model
        model = model_creator.MLP.create_and_load(model_name, input_size=len(feature_columns), output_size=len(target_columns))
        model.eval()
    
        # Perform predictions
        with torch.no_grad():
            y_train_pred = model(x_train)
            y_test_pred = model(x_test)
    
        # Calculate R2 scores for training and testing datasets
        train_r2 = r2_score(y_train.numpy(), y_train_pred.numpy())
        test_r2 = r2_score(y_test.numpy(), y_test_pred.numpy())
    
        # Combine training and testing datasets
        x_combined = torch.cat((x_train, x_test), dim=0)
        y_combined = torch.cat((y_train, y_test), dim=0)
        with torch.no_grad():
            y_combined_pred = model(x_combined)
    
        # Calculate R2 score for the combined dataset
        combined_r2 = r2_score(y_combined.numpy(), y_combined_pred.numpy())
    
        # Print R2 scores
        print(f"Train R2: {train_r2}")
        print(f"Test R2: {test_r2}")
        print(f"Combined R2: {combined_r2}")
    
        # Return R2 scores as a dictionary
        return {"train_r2": train_r2, "test_r2": test_r2, "combined_r2": combined_r2}

    def generate_model_name(self, feature_columns, target_columns, is_standardized, hidden_layers, is_flat=False, prefix="model"):
        """
        Generates a model name based on features, targets, standardization, hidden layers, and whether the data is flattened.

        Args:
            feature_columns (list): List of feature column names.
            target_columns (list): List of target column names.
            is_standardized (bool): Whether the data is standardized.
            hidden_layers (list): List of integers representing the number of neurons in each hidden layer.
            is_flat (bool): Whether the data is flattened. Default is False.
            prefix (str): Optional prefix for the model name. Default is "model".

        Returns:
            str: Generated model name.
        """
        # Join feature and target column names
        features_part = "_".join(feature_columns)
        targets_part = "_".join(target_columns)
        
        # Add standardization information
        standardization_part = "standardized" if is_standardized else ""
        
        # Add flat information only if the model is flat
        flat_part = "_flat" if is_flat else ""
        
        # Add hidden layer information
        hidden_layers_part = "_".join(map(str, hidden_layers))
        
        # Combine all parts into the model name
        model_name = f"{prefix}_{features_part}_to_{targets_part}_{standardization_part}{flat_part}_layers_{hidden_layers_part}.pth"
        return model_name

    def flat_test_r2_standarized(self, x_Scaler_name, y_Scaler_name, model_name="default", feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T']):
        # Load the model
        model = model_creator.MLP.create_and_load(model_name, input_size=len(feature_columns)*71, output_size=len(target_columns))
        model.eval()
    
        # Load the data
        data = self.flat_return_as_standardized_tensors(feature_columns, target_columns,x_Scaler_name,y_Scaler_name)
        features_tr, features_te, targets_tr, targets_te = data
    
        # Perform predictions
        with torch.no_grad():
            pred_tr = model(features_tr)
            pred_te = model(features_te)
    
        # Calculate R2 scores for each target column
        train_r2_scores = {}
        test_r2_scores = {}
    
        for i, target in enumerate(target_columns):
            pred_tr_col = pred_tr[:, i].tolist()
            pred_te_col = pred_te[:, i].tolist()
            target_tr_col = targets_tr[:, i].tolist()
            target_te_col = targets_te[:, i].tolist()
    
            train_r2_scores[target] = r2_score(pred_tr_col, target_tr_col)
            test_r2_scores[target] = r2_score(pred_te_col, target_te_col)
    
        # Print R2 scores
        for target in target_columns:
            print(f"{target} Train R2: {train_r2_scores[target]}")
            print(f"{target} Test R2: {test_r2_scores[target]}")

    def flat_test_r2(self, model_name="default", feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T']):
        # Load the model
        model = model_creator.MLP.create_and_load(model_name, input_size=len(feature_columns)*71, output_size=len(target_columns))
        model.eval()
    
        # Load the data
        data = self.flat_return_as_tensors(feature_columns, target_columns)
        features_tr, features_te, targets_tr, targets_te = data
    
        # Perform predictions
        with torch.no_grad():
            pred_tr = model(features_tr)
            pred_te = model(features_te)

        # Calculate R2 scores for each target column
        train_r2_scores = {}
        test_r2_scores = {}
    
        for i, target in enumerate(target_columns):
            pred_tr_col = pred_tr[:, i].tolist()
            pred_te_col = pred_te[:, i].tolist()
            target_tr_col = targets_tr[:, i].tolist()
            target_te_col = targets_te[:, i].tolist()
    
            train_r2_scores[target] = r2_score(pred_tr_col, target_tr_col)
            test_r2_scores[target] = r2_score(pred_te_col, target_te_col)
    
        # Print R2 scores
        for target in target_columns:
            print(f"{target} Train R2: {train_r2_scores[target]}")
            print(f"{target} Test R2: {test_r2_scores[target]}")


    def flat_test_r2_1000(self, model_name="default", feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T']):
        # Load the model
        model = model_creator.MLP.create_and_load(model_name, input_size=len(feature_columns)*71, output_size=len(target_columns))
        model.eval()
    
        # Load the data
        data = self.flat_return_as_tensors_1000(feature_columns, target_columns)
        features_tr, features_te, targets_tr, targets_te = data
    
        # Perform predictions
        with torch.no_grad():
            pred_tr = model(features_tr)
            pred_te = model(features_te)

        # Calculate R2 scores for each target column
        train_r2_scores = {}
        test_r2_scores = {}
    
        for i, target in enumerate(target_columns):
            pred_tr_col = pred_tr[:, i].tolist()
            pred_te_col = pred_te[:, i].tolist()
            target_tr_col = targets_tr[:, i].tolist()
            target_te_col = targets_te[:, i].tolist()
    
            train_r2_scores[target] = r2_score(pred_tr_col, target_tr_col)
            test_r2_scores[target] = r2_score(pred_te_col, target_te_col)
    
        # Print R2 scores
        for target in target_columns:
            print(f"{target} Train R2: {train_r2_scores[target]}")
            print(f"{target} Test R2: {test_r2_scores[target]}")


    def flat_test_r2_100000(self, model_name="default", feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T']):
        # Load the model
        model = model_creator.MLP.create_and_load(model_name, input_size=len(feature_columns)*71, output_size=len(target_columns))
        model.eval()
    
        # Load the data
        data = self.flat_return_as_tensors_100000(feature_columns, target_columns)
        features_tr, features_te, targets_tr, targets_te = data
    
        # Perform predictions
        with torch.no_grad():
            pred_tr = model(features_tr)
            pred_te = model(features_te)

    
        # Calculate R2 scores for each target column
        train_r2_scores = {}
        test_r2_scores = {}

        for i, target in enumerate(target_columns):
            pred_tr_col = pred_tr[:, i].tolist()
            pred_te_col = pred_te[:, i].tolist()
            target_tr_col = targets_tr[:, i].tolist()
            target_te_col = targets_te[:, i].tolist()
    
            train_r2_scores[target] = r2_score(pred_tr_col, target_tr_col)
            test_r2_scores[target] = r2_score(pred_te_col, target_te_col)
    
        # Print R2 scores
        for target in target_columns:
            print(f"{target} Train R2: {train_r2_scores[target]}")
            print(f"{target} Test R2: {test_r2_scores[target]}")

    def df_split(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42):
        # Splits the dataset into features and targets.
        df = pd.DataFrame()

        for sample in self.samples:
            df = pd.concat([df, sample.data[feature_columns + target_columns]], ignore_index=True)

        df.columns = feature_columns + target_columns
        features = df[feature_columns]
        targets = df[target_columns]
        x_train, x_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

        return [ x_train, x_test, y_train, y_test]

    def flat_df_split(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42):
        # Splits the dataset into features and targets.
        dfFeatures = pd.DataFrame()
        dfTargets = pd.DataFrame()

        for sample in self.samples:
            data = sample.return_as_flat_df(feature_columns, target_columns)
            dfFeatures = pd.concat([dfFeatures, data[0]], ignore_index=True)
            dfTargets = pd.concat([dfTargets, data[1]], ignore_index=True)

        
            
        x_train, x_test, y_train, y_test = train_test_split(dfFeatures, dfTargets, test_size=0.2, random_state=42)

        return [ x_train, x_test, y_train, y_test]


    def poly_lin_reg(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42,degree = 2):
        # Split the dataset into training and testing sets
        data = self.df_split(feature_columns, target_columns, test_size, random_state)
        
        # Initialize polynomial feature transformer and linear regression model
        poly_transformer = PolynomialFeatures(degree=degree)
        lin_reg = LinearRegression()
        
        # Transform features into polynomial features
        polyXtrain = poly_transformer.fit_transform(data[0])
        polyXtest = poly_transformer.transform(data[1])
        
        # Fit the linear regression model on the training data
        lin_reg.fit(polyXtrain, data[2])
        
        # Predict on train and test sets
        pred_train = lin_reg.predict(polyXtrain)
        pred_test = lin_reg.predict(polyXtest)
        
        # Calculate R² coefficients
        r2_train = r2_score(data[2], pred_train)
        r2_test = r2_score(data[3], pred_test)
        
        # Print R² coefficients
        print(f"Train R²: {r2_train}")
        print(f"Test R²: {r2_test}")
        
        # Return R² coefficients as a dictionary
        return {"train_r2": r2_train, "test_r2": r2_test}

    def flat_poly_lin_reg(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42,degree = 2):
        # Split the dataset into training and testing sets
        data = self.flat_df_split(feature_columns, target_columns, test_size, random_state)
        
        # Initialize polynomial feature transformer and linear regression model
        poly_transformer = PolynomialFeatures(degree=degree)
        lin_reg = LinearRegression()
        
        # Transform features into polynomial features
        polyXtrain = poly_transformer.fit_transform(data[0])
        polyXtest = poly_transformer.transform(data[1])
        
        # Fit the linear regression model on the training data
        lin_reg.fit(polyXtrain, data[2])
        
        # Predict on train and test sets
        pred_train = lin_reg.predict(polyXtrain)
        pred_test = lin_reg.predict(polyXtest)
        
        # Calculate R² coefficients
        r2_train = r2_score(data[2], pred_train)
        r2_test = r2_score(data[3], pred_test)
        
        # Print R² coefficients
        print(f"Train R²: {r2_train}")
        print(f"Test R²: {r2_test}")
        
        # Return R² coefficients as a dictionary
        return {"train_r2": r2_train, "test_r2": r2_test}

    def decode_model_name(model_name):
        """
        Decodes a model name to extract the number of input neurons, output neurons, 
        hidden layer neurons, names of output variables, and input variables.
    
        Args:
            model_name (str): The model name to decode.
    
        Returns:
            dict: A dictionary containing the decoded information.
        """
        # Remove the file extension
        model_name = model_name.replace(".pth", "")
        
        # Split the model name into parts
        parts = model_name.split("_")
        
        # Extract input and output variable names
        input_start = parts.index("model") + 1
        to_index = parts.index("to")
        feature_columns = parts[input_start:to_index]
        
        output_start = to_index + 1
        standardization_index = parts.index("standardized") if "standardized" in parts else parts.index("non_standardized")
        target_columns = parts[output_start:standardization_index]
        
        # Extract hidden layers
        layers_index = parts.index("layers") + 1
        hidden_layers = list(map(int, parts[layers_index:]))
        
        # Calculate the number of input and output neurons
        input_neurons = len(feature_columns)
        output_neurons = len(target_columns)
    
        # Return the decoded information
        return {
            "input_neurons": input_neurons,
            "output_neurons": output_neurons,
            "hidden_layers": hidden_layers,
            "input_variables": feature_columns,
            "output_variables": target_columns
        }


    def lin_reg(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42):
        # Split the dataset into training and testing sets
        data = self.df_split(feature_columns, target_columns, test_size, random_state)
        
        # Initialize linear regression model
        lin_reg = LinearRegression()
        
        # Fit the linear regression model on the training data
        lin_reg.fit(data[0], data[2])
        
        # Predict on train and test sets
        pred_train = lin_reg.predict(data[0])
        pred_test = lin_reg.predict(data[1])
        
        # Calculate R² coefficients
        r2_train = r2_score(data[2], pred_train)
        r2_test = r2_score(data[3], pred_test)
        
        # Print R² coefficients
        print(f"Train R²: {r2_train}")
        print(f"Test R²: {r2_test}")
        print(f"Coefficients: {lin_reg.coef_}")

        
        # Return R² coefficients as a dictionary
        return {"train_r2": r2_train, "test_r2": r2_test, "coefficients": lin_reg.coef_}


    def flat_lin_reg(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42):
        # Split the dataset into training and testing sets
        data = self.flat_df_split(feature_columns, target_columns, test_size, random_state)
        
        # Initialize linear regression model
        lin_reg = LinearRegression()
        
        # Fit the linear regression model on the training data
        lin_reg.fit(data[0], data[2])
        
        # Predict on train and test sets
        pred_train = lin_reg.predict(data[0])
        pred_test = lin_reg.predict(data[1])
        
        # Calculate R² coefficients
        r2_train = r2_score(data[2], pred_train)
        r2_test = r2_score(data[3], pred_test)

        coefficients = lin_reg.coef_
        
        # Print R² coefficients
        print(f"Train R²: {r2_train}")
        print(f"Test R²: {r2_test}")
        print(f"Coefficients: {coefficients}")
        
        
        # Return R² coefficients as a dictionary
        return {"train_r2": r2_train, "test_r2": r2_test, "coefficients": coefficients}


    def lasso_reg(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42, alpha = 0.1):
        # Split the dataset into training and testing sets
        data = self.df_split(feature_columns, target_columns, test_size, random_state)
        
        # Initialize Lasso regression model
        lasso_reg = Lasso(alpha=alpha)
        
        # Fit the Lasso regression model on the training data
        lasso_reg.fit(data[0], data[2])
        
        # Predict on train and test sets
        pred_train = lasso_reg.predict(data[0])
        pred_test = lasso_reg.predict(data[1])
        
        # Calculate R² coefficients
        r2_train = r2_score(data[2], pred_train)
        r2_test = r2_score(data[3], pred_test)
        
        # Print R² coefficients
        print(f"Train R²: {r2_train}")
        print(f"Test R²: {r2_test}")
        
        # Return R² coefficients as a dictionary
        return {"train_r2": r2_train, "test_r2": r2_test}


    def flat_lasso_reg(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42, alpha = 0.1):
        # Split the dataset into training and testing sets
        data = self.flat_df_split(feature_columns, target_columns, test_size, random_state)
        
        # Initialize Lasso regression model
        lasso_reg = Lasso(alpha=alpha)
        
        # Fit the Lasso regression model on the training data
        lasso_reg.fit(data[0], data[2])
        
        # Predict on train and test sets
        pred_train = lasso_reg.predict(data[0])
        pred_test = lasso_reg.predict(data[1])
        
        # Calculate R² coefficients
        r2_train = r2_score(data[2], pred_train)
        r2_test = r2_score(data[3], pred_test)
        
        # Print R² coefficients
        print(f"Train R²: {r2_train}")
        print(f"Test R²: {r2_test}")
        
        # Return R² coefficients as a dictionary
        return {"train_r2": r2_train, "test_r2": r2_test, "coefficients": lasso_reg.coef_}


    def ridge_reg(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42, alpha = 0.1):
        # Split the dataset into training and testing sets
        data = self.df_split(feature_columns, target_columns, test_size, random_state)
        
        # Initialize Ridge regression model
        ridge_reg = Ridge(alpha=alpha)
        
        # Fit the Ridge regression model on the training data
        ridge_reg.fit(data[0], data[2])
        
        # Predict on train and test sets
        pred_train = ridge_reg.predict(data[0])
        pred_test = ridge_reg.predict(data[1])
        
        # Calculate R² coefficients
        r2_train = r2_score(data[2], pred_train)
        r2_test = r2_score(data[3], pred_test)
        
        # Print R² coefficients
        print(f"Train R²: {r2_train}")
        print(f"Test R²: {r2_test}")
        
        # Return R² coefficients as a dictionary
        return {"train_r2": r2_train, "test_r2": r2_test}

    def flat_ridge_reg(self, feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], target_columns=['T'], test_size=0.2, random_state = 42, alpha = 0.1):
        # Split the dataset into training and testing sets
        data = self.flat_df_split(feature_columns, target_columns, test_size, random_state)
        
        # Initialize Ridge regression model
        ridge_reg = Ridge(alpha=alpha)
        
        # Fit the Ridge regression model on the training data
        ridge_reg.fit(data[0], data[2])
        
        # Predict on train and test sets
        pred_train = ridge_reg.predict(data[0])
        pred_test = ridge_reg.predict(data[1])
        
        pred_test_df = pd.DataFrame(pred_test)
        pred_test_df.to_excel("predictions.xlsx", index=False)

        data2_df = pd.DataFrame(data[3])
        data2_df.to_excel("predictions1.xlsx", index=False)
        

        
        # Calculate R² coefficients
        r2_train = r2_score(data[2], pred_train)
        r2_test = r2_score(data[3], pred_test)
        
        # Print R² coefficients
        print(f"Train R²: {r2_train}")
        print(f"Test R²: {r2_test}")
        
        # Return R² coefficients as a dictionary
        return {"train_r2": r2_train, "test_r2": r2_test}


    def export_tensor_to_text(self, tensor, file_name, delimiter=","):
        """
        Exports a PyTorch tensor or NumPy array to a text file.
    
        Parameters:
        - tensor (torch.Tensor or np.ndarray): The data to export.
        - file_name (str): The name of the text file to save.
        - delimiter (str): The delimiter to use in the text file. Default is ",".
        """
        # Check if the input is a PyTorch tensor and convert it to a NumPy array
        if isinstance(tensor, torch.Tensor):
            tensor = tensor.numpy()
    
        # Save the NumPy array to a text file
        np.savetxt(file_name, tensor, delimiter=delimiter, fmt="%.6f")
        print(f"Data exported to {file_name}")




        


    def flat_train_1000(self,model_name = "default",
                                feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], 
                                target_columns = ['T'], hidden_layers = [128, 64, 32], loss = nn.MSELoss(), bs = 0):
        """
        Trains a model using flattened data and saves the trained model.
        
        Parameters:
        - model_name: str, the name of the model to save.
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        - hidden_layers: list, the number of neurons in each hidden layer.
        - loss: torch.nn.Module, the loss function to use.
        - save_folder: str, the folder to save the trained model.
        """
        data = self.flat_return_as_tensors_1000(feature_columns, target_columns)
    
        if model_name == "default":
            model_name = self.generate_model_name(feature_columns, target_columns, is_standardized=True, 
                                                  hidden_layers=hidden_layers, is_flat=True)
    
        model = model_creator.MLP(input_size=len(feature_columns)*71, output_size=len(target_columns), hidden_layers=hidden_layers)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay = 0.01)
        model_training.train_model(model, loss, optimizer, data[0], data[2], data[1], data[3], model_name, batch_size=bs)

    def flat_train_100000(self,model_name = "default",
                                feature_columns=['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75'], 
                                target_columns = ['T'], hidden_layers = [128, 64, 32], loss = nn.MSELoss(), bs = 0):
        """
        Trains a model using flattened data and saves the trained model.
        
        Parameters:
        - model_name: str, the name of the model to save.
        - feature_columns: list, the names of the feature columns to include.
        - target_columns: list, the names of the target columns to include.
        - hidden_layers: list, the number of neurons in each hidden layer.
        - loss: torch.nn.Module, the loss function to use.
        - save_folder: str, the folder to save the trained model.
        """
        data = self.flat_return_as_tensors_100000(feature_columns, target_columns)
    
        if model_name == "default":
            model_name = self.generate_model_name(feature_columns, target_columns, is_standardized=True, 
                                                  hidden_layers=hidden_layers, is_flat=True)
        #print(data[2])
        model = model_creator.MLP(input_size=len(feature_columns)*71, output_size=len(target_columns), hidden_layers=hidden_layers)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001, weight_decay = 0.01)
        model_training.train_model(model, loss, optimizer, data[0], data[2], data[1], data[3], model_name, batch_size=bs)